In [ ]:
#workdir_ = '/glade/work/juliob/'
import sys
#######################################
# Leave this for now. But it should change to better
# method as here:
import os
This_module_path = os.getcwd()  #os.path.dirname(os.path.abspath(__file__))
workdir_ = os.path.join(This_module_path, '../../' )
# sys.path.append(utils_path)
# print( f" a path added in {__name__} {utils_path} ")

print( f" In {__name__} we have This_module_path={This_module_path} " )
print( f" In {__name__} we have workdir_={workdir_} " )
########################################
sys.path.append(workdir_ + 'myPythonTools/GenlTools/')
sys.path.append(workdir_ + 'myPythonTools/Utils/')
sys.path.append(workdir_ + 'myPythonTools/Plotting/')
sys.path.append(workdir_ + 'myPythonTools/CASutils/')
#sys.path.append(workdir_ + 'PyRegridding/Regridder/')
sys.path.append(workdir_ + 'PyRegridding/Utils/')

# # #
# Own local packages
import AveragingUtils as Av
#import VertRegridFlexLL as Vrg  # This is toxic for some reason
import PlotUtil as Pu
import utils as uti
import numerical_utils as nuti
import validation_data as Val
import var_A_x_B as vAB
import MakePressures as MkP
import GridUtils as GrU
import LatLonMaps as LL
import filter_utils as fu

# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Some other useful packages 
import importlib
import copy
import time
import cftime

importlib.reload( LL )
importlib.reload( uti )
importlib.reload( nuti )
importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
importlib.reload(vAB)
importlib.reload(MkP)


In [ ]:
OurMonth = '2004-07'
exp='c153_topfix_ne240pg3_FMTHIST_xic_x02'
#exp='fmthist_MM_control_TEM_2'
B = uti.MakeDict4Exp( exp=exp , user='juliob', subd='regridded' , hsPat='cam.h1i' , ymdPat=f'{OurMonth}-*' ,verbose=True, open_dataset=True, shift_lons=True )

#Q = uti.MakeDict4Exp( exp=exp , user='juliob', subd='regridded' , hsPat='cam.h0' , ymdPat='1994-07*' ,verbose=True, open_dataset=True )
#phis=Q.X.PHIS.values/9.8

In [ ]:
####
C = uti.MakeDict4Exp( exp=exp , user='juliob', subd='regridded' , hsPat='cam.h2i' , ymdPat=f'{OurMonth}-*' ,verbose=True, open_dataset=True , shift_lons=True)

In [ ]:
####
D = uti.MakeDict4Exp( exp=exp , user='juliob', subd='regridded' , hsPat='cam.h3i' , ymdPat=f'{OurMonth}-*' ,verbose=True, open_dataset=True , shift_lons=True)

In [ ]:
D.X

In [ ]:
######################################
# Subsample C in time if necessary
######################################
if (C.X.dims['time'] > B.X.dims['time']):
    C.X = C.X.sel( time=B.X.time )

In [ ]:
########
ps=C.X.PS.values

"""
lon0,lat0=100.,-50.
zoog = np.where( (np.abs(lat_x - lat0)<1)&(np.abs(lon_x - lon0)<1) )
print(zoog[0])
"""

In [ ]:
########################
# Some useful vars
########################

hyai_x=B.X.hyai.values
hybi_x=B.X.hybi.values
hyam_x=B.X.hyam.values
hybm_x=B.X.hybm.values

#ps_x =B.X.PS.values
lon_x=B.X.lon.values
lat_x=B.X.lat.values
lev_x=B.X.lev.values
zlev_x=-7.0*np.log( lev_x/1_000. )


In [ ]:

k_44=10
print(zlev_x[k_44])
k_01=80
print(zlev_x[k_01])
k_20=38
print(zlev_x[k_20])
k_05=68
print(zlev_x[k_05])
k_10=58
print(zlev_x[k_10])


In [ ]:
# Only possible on lat-lon field

og44km=B.X.OMEGA[:, k_44, :, :].values
nt,nz,ny,nx = np.shape( B.X.OMEGA )
ss=np.zeros((nt,ny,nx) )
for t in np.arange( nt ):
    if ( (t%10)==0):
        print( f'time {t}' ) 
    ss[t,:,:]=fu.filterk( B.X.OMEGA[t,k_44,:,:] , kmin=0,kmax=20,dimlon=1 )
og44km = og44km - ss

og1km=B.X.OMEGA[:, k_01, :, :].values
ss=np.zeros((nt,ny,nx) )
for t in np.arange( nt ):
    if ( (t%10)==0):
        print( f'time {t}' ) 
    ss[t,:,:]=fu.filterk( B.X.OMEGA[t,k_01,:,:] , kmin=0,kmax=20,dimlon=1 )
og1km = og1km - ss

og20km=B.X.OMEGA[:, k_20, :, :].values
ss=np.zeros((nt,ny,nx) )
for t in np.arange( nt ):
    if ( (t%10)==0):
        print( f'time {t}' ) 
    ss[t,:,:]=fu.filterk( B.X.OMEGA[t,k_20,:,:] , kmin=0,kmax=20,dimlon=1 )
og20km = og20km - ss

og10km=B.X.OMEGA[:, k_10, :, :].values
ss=np.zeros((nt,ny,nx) )
for t in np.arange( nt ):
    if ( (t%10)==0):
        print( f'time {t}' ) 
og10km = og10km - ss


In [ ]:
###
print(f'hoo ')

In [ ]:
###
print(f'hoo ')

In [ ]:
###
print(f'hoo ')

In [ ]:
###


u1km=B.X.U[:, k_01, :, : ].values
v1km=B.X.V[:, k_01, :, : ].values



In [ ]:
###


u5km=B.X.U[:, k_05, :, : ].values
v5km=B.X.V[:, k_05, :, : ].values



In [ ]:
###


upwp1km=D.X.UPWP_CLUBB[:, k_01, :, : ].values
vpwp1km=D.X.VPWP_CLUBB[:, k_01, :, : ].values
wp1km=D.X.WP2_CLUBB[:, k_01, :, : ].values
thp1km=D.X.THLP2_CLUBB[:, k_01, :, : ].values



In [ ]:
%%time
zeta1km=np.zeros((nt,ny,nx) )
for t in np.arange( nt ):
    if ( (t%10)==0):
        print( f'time {t}' ) 
    zeta1km [t,:,:] = nuti.Sphere_Curl2( u1km[t,:,:] , v1km[t,:,:] , lat_x, lon_x, wrap=True )

In [ ]:
%%time
zeta5km=np.zeros((nt,ny,nx) )
for t in np.arange( nt ):
    if ( (t%10)==0):
        print( f'time {t}' ) 
    zeta5km [t,:,:] = nuti.Sphere_Curl2( u5km[t,:,:] , v5km[t,:,:] , lat_x, lon_x, wrap=True )

In [ ]:
shflx=C.X.SHFLX.values



In [ ]:
print(  2.*np.pi*6371. * np.cos( np.radians( -52. ) ) /360. )

In [ ]:
j0,j1=285,399
print( lat_x[[j0,j1]] )

In [ ]:
from scipy.ndimage import gaussian_filter as GF
from matplotlib.ticker import FuncFormatter

# Conversion function: km to degrees longitude
def km_to_lon(x, km_per_deg=111):
    return x / km_per_deg
# Conversion function: m to degrees longitude
def m_to_lon(x, m_per_deg=68_000):
    return (x / m_per_deg)-180.

# Create a formatter function using the conversion function
def longitude_formatter(x, pos):
    return f'{m_to_lon(x):.1f}°'

# Define tick positions in degrees and convert to km
#tick_positions_deg = [60, 120, 180, 240, 300]
tick_positions_deg = [-120, -60, 0, 60, 120]
tick_positions_m = [(deg+180.)* 68_000 for deg in tick_positions_deg]  # Assuming 68 km per degree


pog44km  = np.sqrt(np.average( (og44km[:,j0:j1,:])**2,axis=1) )
pzeta1km = np.sqrt( np.average( (zeta1km[:,j0:j1,:])**2,axis=1) )
pzeta5km = np.sqrt( np.average( (zeta5km[:,j0:j1,:])**2,axis=1) )
pog1km   = np.sqrt( np.average( (og1km[:,j0:j1,:])**2,axis=1) )
pog10km  = np.sqrt( np.average((og10km[:,j0:j1,:])**2,axis=1) )
pog20km  = np.sqrt( np.average((og20km[:,j0:j1,:])**2,axis=1) )
pshflx   = np.average( (shflx[:,j0:j1,:]),axis=1)
pwp1km   = np.average( (wp1km[:,j0:j1,:]),axis=1)
pthp1km   = np.average( (thp1km[:,j0:j1,:]),axis=1)


xx=np.linspace(0,1.,num=nx)*( 2.*np.pi*6_371_000* np.cos( np.radians( -52. ) ))
#xx=np.linspace(-0.5,0.5,num=nx)*( 2.*np.pi*6_371_000* np.cos( np.radians( -52. ) ))
tt=np.linspace(0,1.,num=nt)*6.*3600.*nt
fig,ax=plt.subplots(1,2,squeeze=False,figsize=(14,6))
#plt.contour(pog44km  ,colors='black',levels=5 )
cn0=ax[0,0].contourf(  xx,tt, GF( pog20km,sigma=[2,5] ), cmap='inferno',levels=np.linspace(0.,.08,num=11) )#, linestyles='solid' )#[-4,-3,-2])#,colors='blue',levels=5 )
ln0=ax[0,0].plot( tt*5 , tt, color='black'  ,  label='5ms$^{-1}$'   )
ln0=ax[0,0].plot( tt*10 , tt, '--', color='black' ,  label='10ms$^{-1}$' )
ln0=ax[0,0].plot( tt*20 , tt, '-.', color='black' ,  label='20ms$^{-1}$' )
ax[0,0].set_xlim( 0,xx.max())
# Set the tick positions and formatter for the x-axis
ax[0, 0].set_xticks(tick_positions_m)
ax[0, 0].xaxis.set_major_formatter(FuncFormatter(longitude_formatter))
ax[0,0].legend()
ax[0,0].set_title( '$\omega (20km)$' )

cn1=ax[0,1].contourf(  xx,tt, GF( pzeta1km,sigma=[2,5] ) ,cmap='inferno',levels=15)# , linestyles='solid' )#[-4,-3,-2])#,colors='blue',levels=5 )
ln1=ax[0,1].plot( tt*5 , tt, color='black' )
ln1=ax[0,1].plot( tt*10 , tt, '--', color='black' )
ln1=ax[0,1].plot( tt*20 , tt, '-.', color='black' )
ax[0,1].set_xlim( 0,xx.max())
# Set the formatter for the x-axis
ax[0, 1].set_xticks(tick_positions_m)
ax[0, 1].xaxis.set_major_formatter(FuncFormatter(longitude_formatter))
ax[0,1].set_title( '$\zeta (5km)$' )


fig.colorbar(cn0, ax=ax[0, 0], orientation='vertical')
fig.colorbar(cn1, ax=ax[0, 1], orientation='vertical')
#plt.contour(pshflx   ,colors='orange')

print( np.max(xx)) 

In [ ]:
print( 6_371_000* np.cos( np.radians( -52. ) ))

In [ ]:

fig,ax=plt.subplots(1,2,squeeze=False,figsize=(14,6) )
ax[0,0].scatter( GF(pzeta5km,sigma=[2,25])[0:,1000:2000].flatten() , GF((pog20km),sigma=[2,5])[:,1000:2000].flatten() ,s=.1)
ax[0,0].set_xlabel( '$\zeta (5km) $')
ax[0,0].set_ylabel( '$\omega (20km) $')
ax[0,1].scatter( GF(pzeta5km,sigma=[2,25])[0:,1000:2000].flatten() , GF((pog10km),sigma=[2,5])[:,1000:2000].flatten() ,s=.1)
ax[0,1].set_xlabel( '$\zeta (5km) $')
ax[0,1].set_ylabel( '$\omega (10km) $')
#plt.plot( np.log(pog20km)[:,1000:2000].flatten())

In [ ]:
import numpy as np
import scipy.ndimage
import matplotlib.pyplot as plt

# Example 2D array
data = np.random.rand(100, 100)

# Define 1D kernels for x and y directions
kernel_x = np.array([1, 2, 1]) / 4  # Normalize so the sum is 1
kernel_y = np.array([1, 4, 6, 4, 1]) / 16  # Normalize so the sum is 1

# Apply the kernel along the x-direction
smoothed_data_x = scipy.ndimage.convolve1d(data, kernel_x, axis=1)

# Apply the kernel along the y-direction
smoothed_data = scipy.ndimage.convolve1d(smoothed_data_x, kernel_y, axis=0)

# Plot the original and smoothed data
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].imshow(data, cmap='viridis')
ax[0].set_title('Original Data')
ax[1].imshow(smoothed_data, cmap='viridis')
ax[1].set_title('Smoothed Data with Separable Kernels')
plt.show()

In [ ]:
og44km_var=np.average( og44km**2 , axis=0)

In [ ]:
zeta5km_var=np.average( zeta5km**2 , axis=0)

In [ ]:
og44km_var.shape

In [ ]:
plt.contourf( np.sqrt(og44km_var) ,levels=21)

In [ ]:

LL.Maps( fields=[np.sqrt(og44km_var )],lons=[lon_x],lats=[lat_x] ,clevs=[21],cmaps=['gist_ncar'])

In [ ]:

LL.Maps( fields=[np.sqrt(zeta5km_var )],lons=[lon_x],lats=[lat_x] ,clevs=[np.linspace(0.,0.0001,num=21)],cmaps=['gist_ncar'])